In [47]:
# import the relevant package and dataset 

from torch_snippets import *
#!wget https://www.dropbox.com/s/ua1rr8btkmpqjxh/face-detection.zip
#!unzip face-detection.zip
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device_used :', device)

device_used : cuda

In [49]:
# Define the dataset class - SiemeseNetworDataset. 
class SiemeseNetworkDataset(Dataset):
    def __init__(self,folder,transform=None, should_invert=True):
        self.folder = folder 
        self.items = Glob(f'{self.folder}/*/*')
        self.transform = transform
    def __len__(self):
        return len(self.items)
    def __getitem__(self, ix):
        itemA = self.items[ix]
        person = fname(parent(itemA))
        same_person = randint(2)
        if same_person: 
            itemB = choose(Glob(f'{self.folder}/{person}/*', silent=True))
        else: 
            while True:
                itemB = choose(self.items)
                if person != fname(parent(itemB)):
                    break
        imgA = read(itemA)
        imgB = read(itemB)
        if self.transform:
            imgA = self.transform(imgA)
            imgB = self.transform(imgB)
        return imgA, imgB, np.array([1-same_person])

In [50]:
from torchvision import transforms 

trn_tfms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(), 
    transforms.RandomAffine(5, (0.01,0.2),scale=(0.9,1.1)),
    transforms.Resize((100,100)),
    transforms.ToTensor(), 
    transforms.Normalize((0.5), (0.5)) 
])

val_tfms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((100,100)),
    transforms.ToTensor(), 
    transforms.Normalize((0.5), (0.5))
])

trn_ds = SiemeseNetworkDataset(folder = './data/faces/training/', transform=trn_tfms)
val_ds = SiemeseNetworkDataset(folder = './data/faces/testing/', transform=val_tfms)

trn_dl = DataLoader(trn_ds, shuffle=True, batch_size=64)
val_dl = DataLoader(val_ds, shuffle=False, batch_size=64)


In [42]:
# testing class or function 
folder = './data/faces/training/'
testing = SiemeseNetworkDataset(folder)
print(testing.__getitem__(5))

(
    array([[122, 123, 122, ..., 118, 114, 118],
       [124, 122, 122, ..., 117, 116, 114],
       [122, 123, 122, ..., 116, 115, 115],
       ...,
       [132, 110, 131, ...,  98,  92,  93],
       [128, 116, 137, ...,  99,  93,  90],
       [126, 121, 146, ...,  98, 101,  85]], dtype=uint8),
    array([[123, 122, 121, ..., 117, 116, 115],
       [124, 123, 121, ..., 115, 116, 117],
       [123, 123, 123, ..., 116, 116, 116],
       ...,
       [ 88, 100, 129, ...,  94,  85,  79],
       [ 86,  95, 129, ...,  84,  92,  95],
       [ 78,  95, 125, ...,  84,  92,  94]], dtype=uint8),
    array([0])
)